In [1]:
import numpy as np 
from matplotlib import pyplot as plt
import tensorflow as tf 
from tensorflow.keras import datasets, layers, models

In [2]:
(train_img, train_label),(test_img, test_label) = datasets.mnist.load_data()

In [3]:
print(f"Training images shape is = {train_img.shape}")
print(f"Training labels shape is = {train_img.shape}")
print(f"Testing images shape is = {test_img.shape}")
print(f"Testing labels shape is = {test_img.shape}")

Training images shape is = (60000, 28, 28)
Training labels shape is = (60000, 28, 28)
Testing images shape is = (10000, 28, 28)
Testing labels shape is = (10000, 28, 28)


In [4]:
train_img = train_img/255.0
test_img = test_img/255.0

In [15]:
lenet = models.Sequential([
  
  layers.Conv2D(filters=6, kernel_size=(2, 2), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
  layers.AveragePooling2D((2,2)),

  layers.Conv2D(filters=16, kernel_size=(2, 2), strides=(2, 2), activation='relu'),
  layers.AveragePooling2D((2,2)),
  
  layers.Flatten(),
  layers.Dense(120, activation='relu'),
  layers.Dense(84, activation='relu'),
  layers.Dense(10, activation='softmax'),
  
])

In [16]:
lenet.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
lenet.fit(train_img, train_label, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4659 - accuracy: 0.8535
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1928 - accuracy: 0.9385
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1449 - accuracy: 0.9539
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1185 - accuracy: 0.9617
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1003 - accuracy: 0.9674
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0868 - accuracy: 0.9724
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0754 - accuracy: 0.9753
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0683 - accuracy: 0.9778
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0618 - accuracy: 0.9797
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.057

In [18]:
lenet.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 27, 27, 6)         30        
                                                                 
 average_pooling2d_4 (Avera  (None, 13, 13, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_7 (Conv2D)           (None, 6, 6, 16)          400       
                                                                 
 average_pooling2d_5 (Avera  (None, 3, 3, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten_2 (Flatten)         (None, 144)               0         
                                                                 
 dense_6 (Dense)             (None, 120)              

In [13]:
lenet.evaluate(test_img, test_label)

313/313 [==============================] - 1s 2ms/step - loss: 0.0408 - accuracy: 0.9883


[0.040806859731674194, 0.9883000254631042]

In [14]:
lenet.save("LeNet-DigitRecognition.h5")

c:\Users\shory\anaconda3\envs\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
